In [2]:
import os
import io
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.applications.resnet50 import ResNet50
from keras import Sequential, layers, optimizers, Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import base64
import csv

In [50]:
# Path:

base_path = "D:/UIT/DACN/"
csv_folder_path = os.path.join(base_path, "train_csv/")
img_folder_path = os.path.join(base_path, "train_img/")
img_path_test = "D:/UIT/DACN/test_img/2024-01-15-11-02-05"
img_path = ""


In [51]:
# Build model

def prepare_model():
    model = Sequential()
    model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224, 224, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
    model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])
    return model

model = prepare_model()


In [52]:
# Function:

def map_rain_value(rain_value):
    if rain_value != 0:
        return 1
    else:
        return 0


def get_csv_file_name(file_path):
    csv_file = os.path.basename(file_path)
    csv_name = csv_file[:-4]
    return csv_name


def find_img_folder_matched(csv_name):
    for img_folder in os.listdir(img_folder_path):
        full_img_folder_path = os.path.join(img_folder_path, img_folder)   
        if os.path.isdir(full_img_folder_path) and img_folder == csv_name:
            matching_img_folder = full_img_folder_path
            break
    return matching_img_folder


def train_data(df, path):
    batch_size = 8
    train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.20)

    train_generator = train_datagen.flow_from_dataframe(
        dataframe=df,
        directory=path,
        x_col="City",
        y_col="Rain (mm)",
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode="categorical",
        subset='training',
        shuffle=True,
        seed=42)

    valid_generator = train_datagen.flow_from_dataframe(
        dataframe=df,
        directory=path,
        x_col="City",
        y_col="Rain (mm)",
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode="categorical",
        subset='validation',
        shuffle=True,
        seed=42)
        
    model.fit(train_generator,
              validation_data = train_generator,
              steps_per_epoch = train_generator.n//train_generator.batch_size,
              validation_steps = valid_generator.n//valid_generator.batch_size,
              epochs=5)


def prepare_data_to_train(file, path):
    file_path = os.path.join(path, file)
    column_names = ["City", "Rain (mm)"]
    df = pd.read_csv(file_path, usecols=column_names)
    df["Rain (mm)"] = df["Rain (mm)"].apply(map_rain_value)
    df['City'] = df['City'].apply(lambda x: x + '.png')
    df['Rain (mm)'] = df['Rain (mm)'].astype(str)
    
    csv_name = get_csv_file_name(file_path)
    img_folder = find_img_folder_matched(csv_name)
    global img_path
    img_path = img_folder
    return df
         

In [54]:
# Train:

for csv_file in os.listdir(csv_folder_path):
    if csv_file.endswith(".csv"):
        df = prepare_data_to_train(csv_file, csv_folder_path)
        train_data(df, img_path)


Found 61 validated image filenames belonging to 1 classes.
Found 15 validated image filenames belonging to 1 classes.


C:\Users\ASUS\AppData\Local\Temp\ipykernel_16956\3368467860.py:59: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/5


7/7 [==============================] - 3s 225ms/step - loss: 0.1335 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/5
7/7 [==============================] - 1s 208ms/step - loss: 1.0524e-34 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/5
7/7 [==============================] - 1s 194ms/step - loss: 1.9717e-36 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/5
7/7 [==============================] - 1s 196ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/5
7/7 [==============================] - 1s 190ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Found 61 validated image filenames belonging to 1 classes.
Found 15 validated image filenames belonging to 1 classes.
Epoch 1/5


C:\Users\ASUS\AppData\Local\Temp\ipykernel_16956\3368467860.py:59: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


7/7 [==============================] - 2s 217ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/5
7/7 [==============================] - 1s 192ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/5
7/7 [==============================] - 1s 193ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/5
7/7 [==============================] - 1s 193ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/5
7/7 [==============================] - 1s 200ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Found 61 validated image filenames belonging to 1 classes.
Found 15 validated image filenames belonging to 1 classes.
Epoch 1/5


...

7/7 [==============================] - 1s 203ms/step - loss: 0.4139 - accuracy: 1.0000 - val_loss: 0.4139 - val_accuracy: 1.0000
Epoch 2/5
7/7 [==============================] - 1s 192ms/step - loss: 0.4134 - accuracy: 1.0000 - val_loss: 0.4126 - val_accuracy: 1.0000
Epoch 3/5
7/7 [==============================] - 1s 194ms/step - loss: 0.4117 - accuracy: 1.0000 - val_loss: 0.4105 - val_accuracy: 1.0000
Epoch 4/5
7/7 [==============================] - 1s 185ms/step - loss: 0.4096 - accuracy: 1.0000 - val_loss: 0.4082 - val_accuracy: 1.0000
Epoch 5/5
7/7 [==============================] - 1s 189ms/step - loss: 0.4071 - accuracy: 1.0000 - val_loss: 0.4057 - val_accuracy: 1.0000
Found 61 validated image filenames belonging to 1 classes.
Found 15 validated image filenames belonging to 1 classes.
Epoch 1/5


C:\Users\ASUS\AppData\Local\Temp\ipykernel_16956\3368467860.py:59: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


7/7 [==============================] - 1s 206ms/step - loss: 0.4046 - accuracy: 1.0000 - val_loss: 0.4031 - val_accuracy: 1.0000
Epoch 2/5
7/7 [==============================] - 1s 195ms/step - loss: 0.4020 - accuracy: 1.0000 - val_loss: 0.4005 - val_accuracy: 1.0000
Epoch 3/5
7/7 [==============================] - 1s 195ms/step - loss: 0.3994 - accuracy: 1.0000 - val_loss: 0.3980 - val_accuracy: 1.0000
Epoch 4/5
7/7 [==============================] - 1s 187ms/step - loss: 0.3969 - accuracy: 1.0000 - val_loss: 0.3954 - val_accuracy: 1.0000
Epoch 5/5
7/7 [==============================] - 1s 198ms/step - loss: 0.3943 - accuracy: 1.0000 - val_loss: 0.3929 - val_accuracy: 1.0000
Found 61 validated image filenames belonging to 1 classes.
Found 15 validated image filenames belonging to 1 classes.
Epoch 1/5


C:\Users\ASUS\AppData\Local\Temp\ipykernel_16956\3368467860.py:59: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


7/7 [==============================] - 1s 207ms/step - loss: 0.3918 - accuracy: 1.0000 - val_loss: 0.3904 - val_accuracy: 1.0000
Epoch 2/5
7/7 [==============================] - 1s 193ms/step - loss: 0.3894 - accuracy: 1.0000 - val_loss: 0.3880 - val_accuracy: 1.0000
Epoch 3/5
7/7 [==============================] - 1s 187ms/step - loss: 0.3870 - accuracy: 1.0000 - val_loss: 0.3855 - val_accuracy: 1.0000
Epoch 4/5
7/7 [==============================] - 1s 195ms/step - loss: 0.3845 - accuracy: 1.0000 - val_loss: 0.3831 - val_accuracy: 1.0000
Epoch 5/5
7/7 [==============================] - 1s 188ms/step - loss: 0.3821 - accuracy: 1.0000 - val_loss: 0.3807 - val_accuracy: 1.0000
Found 76 validated image filenames.


In [82]:
# Evaluate

csv_test_path = "D:/UIT/DACN/test_csv/2024-01-15-11-02-05.csv"
columns_to_keep = ["City", "Rain (mm)"]
test_df = pd.read_csv(csv_test_path, usecols=columns_to_keep)
test_df["Rain (mm)"] = test_df["Rain (mm)"].apply(map_rain_value)
test_df['City'] = test_df['City'].apply(lambda x: x + '.png')
test_df['Rain (mm)'] = test_df['Rain (mm)'].astype(str)

train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.20)

valid_generator = train_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=img_path_test,
        x_col="City",
        y_col="Rain (mm)",
        target_size=(224, 224),
        batch_size=8,
        class_mode="categorical",
        subset='validation',
        shuffle=True,
        seed=42)

score = model.evaluate(valid_generator)
print('\nTest loss:', score[0])
print('Test accuracy:', score[1])


Found 15 validated image filenames belonging to 2 classes.
2/2 [==============================] - 0s 89ms/step - loss: 0.4479 - accuracy: 1.0000

Test loss: 0.44794896245002747
Test accuracy: 1.0


In [80]:
# Predict:

test_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.20)        

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=img_path_test,
    x_col="City",
    target_size=(224, 224),
    batch_size=1,
    class_mode=None,
    shuffle=False,)

predict = model.predict(test_generator, steps=len(test_generator.filenames))

Found 76 validated image filenames.
76/76 [==============================] - 1s 13ms/step


In [62]:
result_predict = predict.argmax(axis=-1)
print(result_predict)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]


In [76]:
real_result = np.array(test_df['Rain (mm)'].astype(int))
print(np.array_str(real_result, precision=0, suppress_small=True))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]


In [79]:
# Classification report

labels = test_df['Rain (mm)'].astype(int).tolist()

predicted_labels = np.argmax(predict, axis=1)

print(classification_report(labels, predicted_labels, zero_division=1))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99        75
           1       1.00      0.00      0.00         1

    accuracy                           0.99        76
   macro avg       0.99      0.50      0.50        76
weighted avg       0.99      0.99      0.98        76

